# Football Data Analysis

This notebook processes football match data to calculate various statistics over different time windows (last 5, 15, and 38 matches) for both home and away teams.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Read the Excel file
df_raw = pd.read_excel('Football Data Test Task.xlsx', sheet_name='Raw Data')
print('Raw data shape:', df_raw.shape)
df_raw.head()

## Calculate Rolling Statistics

We'll calculate rolling statistics for each team over their last 5, 15, and 38 matches. For each statistic, we'll consider both home and away matches.

In [ ]:
def calculate_team_stats(df, team, n_matches):
    """Calculate statistics for a team over their last N matches.
    
    Args:
        df: DataFrame containing match data
        team: Team name to calculate statistics for
        n_matches: Number of previous matches to consider
        
    Returns:
        Dictionary containing calculated statistics
    """
    # Get all matches where the team played (either home or away)
    team_matches = df[
        (df['HomeTeam'] == team) | (df['AwayTeam'] == team)
    ].copy()
    
    # Sort by Incremental_ID to ensure chronological order
    team_matches = team_matches.sort_values('Incremental_ID')
    
    stats = {}
    
    # Calculate goals
    home_goals = team_matches[team_matches['HomeTeam'] == team]['FTHG']
    away_goals = team_matches[team_matches['AwayTeam'] == team]['FTAG']
    goals = pd.concat([home_goals, away_goals]).sort_index()
    stats[f'Goals_L{n_matches}'] = goals.rolling(window=n_matches, min_periods=1).sum()
    
    # Calculate wins
    home_wins = ((team_matches['HomeTeam'] == team) & (team_matches['FTR'] == 'H'))
    away_wins = ((team_matches['AwayTeam'] == team) & (team_matches['FTR'] == 'A'))
    wins = pd.concat([home_wins, away_wins]).sort_index()
    stats[f'Wins_L{n_matches}'] = wins.rolling(window=n_matches, min_periods=1).sum()
    
    # Add more statistics as needed (shots, corners, cards, etc.)
    
    return stats

In [ ]:
# Process a sample team
sample_team = df_raw['HomeTeam'].iloc[0]  # First team in the dataset
stats = calculate_team_stats(df_raw, sample_team, 5)
print(f'Statistics for {sample_team} over last 5 matches:')
for stat, values in stats.items():
    print(f'{stat}: {values.iloc[-1]:.0f}')

## Process All Teams

Now we'll process all teams and create columns for each statistic over different time windows.

In [ ]:
def process_all_teams(df):
    """Process all teams and calculate their statistics."""
    processed_df = df.copy()
    
    # Get unique teams
    all_teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).unique()
    
    # Calculate statistics for each team
    team_stats = {}
    for team in all_teams:
        team_stats[team] = {
            5: calculate_team_stats(df, team, 5),
            15: calculate_team_stats(df, team, 15),
            38: calculate_team_stats(df, team, 38)
        }
    
    # Add statistics to the dataframe
    for idx, row in processed_df.iterrows():
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        
        # Add home team stats
        for n in [5, 15, 38]:
            for stat, values in team_stats[home_team][n].items():
                col_name = f'Home_{stat}'
                processed_df.at[idx, col_name] = values[idx] if idx in values.index else None
        
        # Add away team stats
        for n in [5, 15, 38]:
            for stat, values in team_stats[away_team][n].items():
                col_name = f'Away_{stat}'
                processed_df.at[idx, col_name] = values[idx] if idx in values.index else None
    
    return processed_df

In [ ]:
# Process all teams
processed_df = process_all_teams(df_raw)

# Save to Excel
with pd.ExcelWriter('Football Data Test Task.xlsx', mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
    processed_df.to_excel(writer, sheet_name='Processed Data', index=False)

print('New columns added:')
new_cols = [col for col in processed_df.columns if col not in df_raw.columns]
print('\n'.join(new_cols))

## Verify Calculations

Let's verify our calculations match the example provided:
FTHG_L5 (for Arsenal) = G372 + H370 + H366 + G349 + G332

In [ ]:
# Verify calculations for Arsenal
arsenal_matches = df_raw[
    (df_raw['HomeTeam'] == 'Arsenal') | (df_raw['AwayTeam'] == 'Arsenal')
].sort_values('Incremental_ID').tail(5)

print('Last 5 matches for Arsenal:')
print(arsenal_matches[['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG']])

# Calculate total goals
total_goals = sum([
    row['FTHG'] if row['HomeTeam'] == 'Arsenal' else row['FTAG']
    for _, row in arsenal_matches.iterrows()
])

print(f'\nTotal goals in last 5 matches: {total_goals}')